In [1]:
import random
import os
import re
import unicodedata
import zipfile
 
import requests
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tokenizers
import tqdm

In [2]:
import requests

url = "https://www.manythings.org/anki/pol-eng.zip"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/126.0 Safari/537.36"
}

r = requests.get(url, headers=headers)
r.raise_for_status()

with open("pol-eng.zip", "wb") as f:
    f.write(r.content)


In [3]:

import os
import unicodedata
import zipfile
import requests

url = "https://www.manythings.org/anki/pol-eng.zip"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/126.0 Safari/537.36"
}

r = requests.get(url, headers=headers)
r.raise_for_status()

with open("pol-eng.zip", "wb") as f:
    f.write(r.content)

In [4]:
# Normalize text
# each line of the file is in the format "<english>\t<french>"
# We convert text to lowercase, normalize unicode (UFKC)
def normalize(line):
    """Normalize a line of text and split into two at the tab character"""
    line = unicodedata.normalize("NFKC", line.strip().lower())
    parts = line.split("\t")
    if len(parts) < 2:
        return None  # pomiń niepoprawne linie

    # niektóre linie mają więcej niż jedno tłumaczenie – weź tylko pierwsze dwa
    eng, pl = parts[0], parts[1]

    return eng.strip(), pl.strip()

text_pairs = []
with zipfile.ZipFile("pol-eng.zip", "r") as zip_ref:
    for line in zip_ref.read("pol.txt").decode("utf-8").splitlines():
        eng, pol = normalize(line)
        text_pairs.append((eng, pol))

In [5]:
import tokenizers
 
if os.path.exists("en_tokenizer.json") and os.path.exists("pl_tokenizer.json"):
    en_tokenizer = tokenizers.Tokenizer.from_file("en_tokenizer.json")
    pl_tokenizer = tokenizers.Tokenizer.from_file("pl_tokenizer.json")
else:
    en_tokenizer = tokenizers.Tokenizer(tokenizers.models.BPE())
    pl_tokenizer = tokenizers.Tokenizer(tokenizers.models.BPE())
 
    # Configure pre-tokenizer to split on whitespace and punctuation, add space at beginning of the sentence
    en_tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.ByteLevel(add_prefix_space=True)
    pl_tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.ByteLevel(add_prefix_space=True)
 
    # Configure decoder: So that word boundary symbol "Ġ" will be removed
    en_tokenizer.decoder = tokenizers.decoders.ByteLevel()
    pl_tokenizer.decoder = tokenizers.decoders.ByteLevel()
 
    # Train BPE for English and French using the same trainer
    VOCAB_SIZE = 8000
    trainer = tokenizers.trainers.BpeTrainer(
        vocab_size=VOCAB_SIZE,
        special_tokens=["[start]", "[end]", "[pad]"],
        show_progress=True
    )
    en_tokenizer.train_from_iterator([x[0] for x in text_pairs], trainer=trainer)
    pl_tokenizer.train_from_iterator([x[1] for x in text_pairs], trainer=trainer)
 
    en_tokenizer.enable_padding(pad_id=en_tokenizer.token_to_id("[pad]"), pad_token="[pad]")
    pl_tokenizer.enable_padding(pad_id=pl_tokenizer.token_to_id("[pad]"), pad_token="[pad]")
 
    # Save the trained tokenizers
    en_tokenizer.save("en_tokenizer.json", pretty=True)
    pl_tokenizer.save("pl_tokenizer.json", pretty=True)

In [7]:
def rotate_half(x):
    x1, x2 = x.chunk(2, dim=-1)
    return torch.cat((-x2, x1), dim=-1)

def apply_rotary_pos_emb(x, cos, sin):
    return (x * cos) + (rotate_half(x) * sin)

class RotaryPositionalEncoding(nn.Module):
    def __init__(self, dim, max_seq_len=1024):
        super().__init__()
        N = 10000
        inv_freq = 1. / (N ** (torch.arange(0, dim, 2).float() / dim))
        position = torch.arange(max_seq_len).float()
        inv_freq = torch.cat((inv_freq, inv_freq), dim=-1)
        sinusoid_inp = torch.outer(position, inv_freq)
        self.register_buffer("cos", sinusoid_inp.cos())
        self.register_buffer("sin", sinusoid_inp.sin())

    def forward(self, x, seq_len=None):
        if seq_len is None:
            seq_len = x.size(1)
        cos = self.cos[:seq_len].view(1, seq_len, 1, -1)
        sin = self.sin[:seq_len].view(1, seq_len, 1, -1)
        return apply_rotary_pos_emb(x, cos, sin)

In [8]:
class GQA(nn.Module):
    def __init__(self, hidden_dim, num_heads, num_kv_heads=None, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.num_kv_heads = num_kv_heads or num_heads
        self.head_dim = hidden_dim // num_heads
        self.num_groups = num_heads // num_kv_heads
        self.dropout = dropout
        self.q_proj = nn.Linear(hidden_dim, hidden_dim)
        self.k_proj = nn.Linear(hidden_dim, hidden_dim)
        self.v_proj = nn.Linear(hidden_dim, hidden_dim)
        self.out_proj = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, q, k, v, mask=None, rope=None):
        q_batch_size, q_seq_len, hidden_dim = q.shape
        k_batch_size, k_seq_len, hidden_dim = k.shape
        v_batch_size, v_seq_len, hidden_dim = v.shape

        # projection
        q = self.q_proj(q).view(q_batch_size, q_seq_len, -1, self.head_dim).transpose(1, 2)
        k = self.k_proj(k).view(k_batch_size, k_seq_len, -1, self.head_dim).transpose(1, 2)
        v = self.v_proj(v).view(v_batch_size, v_seq_len, -1, self.head_dim).transpose(1, 2)

        # apply rotary positional encoding
        if rope:
            q = rope(q)
            k = rope(k)

        # compute grouped query attention
        q = q.contiguous()
        k = k.contiguous()
        v = v.contiguous()
        output = F.scaled_dot_product_attention(q, k, v,
                                                attn_mask=mask,
                                                dropout_p=self.dropout,
                                                enable_gqa=True)
        output = output.transpose(1, 2).reshape(q_batch_size, q_seq_len, hidden_dim).contiguous()
        output = self.out_proj(output)
        return output

In [9]:
class SwiGLU(nn.Module):
    def __init__(self, hidden_dim, intermediate_dim):
        super().__init__()
        self.gate = nn.Linear(hidden_dim, intermediate_dim)
        self.up = nn.Linear(hidden_dim, intermediate_dim)
        self.down = nn.Linear(intermediate_dim, hidden_dim)
        self.act = nn.SiLU()

    def forward(self, x):
        x = self.act(self.gate(x)) * self.up(x)
        x = self.down(x)
        return x

In [10]:
class EncoderLayer(nn.Module):
    def __init__(self, hidden_dim, num_heads, num_kv_heads=None, dropout=0.1):
        super().__init__()
        self.self_attn = GQA(hidden_dim, num_heads, num_kv_heads, dropout)
        self.mlp = SwiGLU(hidden_dim, 4 * hidden_dim)
        self.norm1 = nn.RMSNorm(hidden_dim)
        self.norm2 = nn.RMSNorm(hidden_dim)

    def forward(self, x, mask=None, rope=None):
        # self-attention sublayer
        out = x
        out = self.norm1(x)
        out = self.self_attn(out, out, out, mask, rope)
        x = out + x
        # MLP sublayer
        out = self.norm2(x)
        out = self.mlp(out)
        return out + x

In [11]:
class DecoderLayer(nn.Module):
    def __init__(self, hidden_dim, num_heads, num_kv_heads=None, dropout=0.1):
        super().__init__()
        self.self_attn = GQA(hidden_dim, num_heads, num_kv_heads, dropout)
        self.cross_attn = GQA(hidden_dim, num_heads, num_kv_heads, dropout)
        self.mlp = SwiGLU(hidden_dim, 4 * hidden_dim)
        self.norm1 = nn.RMSNorm(hidden_dim)
        self.norm2 = nn.RMSNorm(hidden_dim)
        self.norm3 = nn.RMSNorm(hidden_dim)

    def forward(self, x, enc_out, mask=None, rope=None):
        # self-attention sublayer
        out = x
        out = self.norm1(out)
        out = self.self_attn(out, out, out, mask, rope)
        x = out + x
        # cross-attention sublayer
        out = self.norm2(x)
        out = self.cross_attn(out, enc_out, enc_out, None, rope)
        x = out + x
        # MLP sublayer
        x = out + x
        out = self.norm3(x)
        out = self.mlp(out)
        return out + x

In [12]:
class Transformer(nn.Module):
    def __init__(self, num_layers, num_heads, num_kv_heads, hidden_dim,
                 max_seq_len, vocab_size_src, vocab_size_tgt, dropout=0.1):
        super().__init__()
        self.rope = RotaryPositionalEncoding(hidden_dim // num_heads, max_seq_len)
        self.src_embedding = nn.Embedding(vocab_size_src, hidden_dim)
        self.tgt_embedding = nn.Embedding(vocab_size_tgt, hidden_dim)
        self.encoders = nn.ModuleList([
            EncoderLayer(hidden_dim, num_heads, num_kv_heads, dropout) for _ in range(num_layers)
        ])
        self.decoders = nn.ModuleList([
            DecoderLayer(hidden_dim, num_heads, num_kv_heads, dropout) for _ in range(num_layers)
        ])
        self.out = nn.Linear(hidden_dim, vocab_size_tgt)

    def forward(self, src_ids, tgt_ids, src_mask=None, tgt_mask=None):
        # Encoder
        x = self.src_embedding(src_ids)
        for encoder in self.encoders:
            x = encoder(x, src_mask, self.rope)
        enc_out = x
        # Decoder
        x = self.tgt_embedding(tgt_ids)
        for decoder in self.decoders:
            x = decoder(x, enc_out, tgt_mask, self.rope)
        return self.out(x)

In [13]:
model_config = {
    "num_layers": 4,
    "num_heads": 8,
    "num_kv_heads": 4,
    "hidden_dim": 128,
    "max_seq_len": 768,
    "vocab_size_src": len(en_tokenizer.get_vocab()),
    "vocab_size_tgt": len(pl_tokenizer.get_vocab()),
    "dropout": 0.1,
}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Transformer(**model_config).to(device)

In [14]:
import torch
from torch.utils.data import Dataset, DataLoader

class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, text_pairs):
        self.text_pairs = text_pairs

    def __len__(self):
        return len(self.text_pairs)

    def __getitem__(self, idx):
        eng, pol = self.text_pairs[idx]
        return eng, "[start] " + pol + " [end]"


def collate_fn(batch):
    en_str, pl_str = zip(*batch)
    en_enc = en_tokenizer.encode_batch(en_str, add_special_tokens=True)
    pl_enc = pl_tokenizer.encode_batch(pl_str, add_special_tokens=True)
    en_ids = [enc.ids for enc in en_enc]
    pl_ids = [enc.ids for enc in pl_enc]
    return torch.tensor(en_ids), torch.tensor(pl_ids)

BATCH_SIZE = 32
dataset = TranslationDataset(text_pairs)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

In [15]:
for en_ids, pl_ids in dataloader:
    print(f"English: {en_ids}")
    print(f"French: {pl_ids}")
    break

English: tensor([[ 261,  286,   74, 2352,  127,   24,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2],
        [  61,  168,  111,  147,  105,  113,   11,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2],
        [  93,  273,  796,  239, 2080,  153, 3421,   11,    2,    2,    2,    2,
            2,    2,    2,    2,    2],
        [ 243,  121,   75, 4655,  305,  232,   24,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2],
        [ 114,  387,   92,  406,   11,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2],
        [ 113,  121,   64, 1164, 2778,   11,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2],
        [  80, 1343,   92,  320,  372,  114,   93,  121, 5094,   65,  105,  114,
          681,   11,    2,    2,    2],
        [ 812,  637,  127,   11,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2],
        [  61,  142,   

In [16]:
def create_causal_mask(seq_len, device):
    mask = torch.triu(torch.full((seq_len, seq_len), float('-inf'), device=device), diagonal=1)
    return mask

In [17]:
def create_padding_mask(batch, padding_token_id):
    batch_size, seq_len = batch.shape
    device = batch.device
    padded = torch.zeros_like(batch, device=device).float().masked_fill(batch == padding_token_id, float('-inf'))
    mask = torch.zeros(batch_size, seq_len, seq_len, device=device) + padded[:,:,None] + padded[:,None,:]
    return mask[:, None, :, :]

In [18]:
N_EPOCHS = 60
LR = 0.005
WARMUP_STEPS = 1000
CLIP_NORM = 5.0
best_loss = float('inf')

loss_fn = nn.CrossEntropyLoss(ignore_index=pl_tokenizer.token_to_id("[pad]"))

optimizer = optim.Adam(model.parameters(), lr=LR)
warmup_scheduler = optim.lr_scheduler.LinearLR(
    optimizer, start_factor=0.01, end_factor=1.0, total_iters=WARMUP_STEPS)
cosine_scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=N_EPOCHS * len(dataloader) - WARMUP_STEPS, eta_min=0)
scheduler = optim.lr_scheduler.SequentialLR(
    optimizer, schedulers=[warmup_scheduler, cosine_scheduler], milestones=[WARMUP_STEPS])

for epoch in range(N_EPOCHS):
    model.train()
    epoch_loss = 0
    for en_ids, pl_ids in dataloader:
        # Move the "sentences" to device
        en_ids = en_ids.to(device)
        pl_ids = pl_ids.to(device)
        # create source mask as padding mask, target mask as causal mask
        src_mask = create_padding_mask(en_ids, en_tokenizer.token_to_id("[pad]"))
        tgt_mask = create_causal_mask(pl_ids.shape[1], device).unsqueeze(0)
        tgt_mask = tgt_mask + create_padding_mask(pl_ids, pl_tokenizer.token_to_id("[pad]"))
        # zero the grad, then forward pass
        optimizer.zero_grad()
        outputs = model(en_ids, pl_ids, src_mask, tgt_mask)
        # compute the loss: compare 3D logits to 2D targets
        loss = loss_fn(outputs[:, :-1, :].reshape(-1, outputs.shape[-1]), pl_ids[:, 1:].reshape(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP_NORM, error_if_nonfinite=False)
        optimizer.step()
        scheduler.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{N_EPOCHS}; Avg loss {epoch_loss/len(dataloader)}; Latest loss {loss.item()}")
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for en_ids, pl_ids in tqdm.tqdm(dataloader, desc="Evaluating"):
            en_ids = en_ids.to(device)
            pl_ids = pl_ids.to(device)
            src_mask = create_padding_mask(en_ids, en_tokenizer.token_to_id("[pad]"))
            tgt_mask = create_causal_mask(pl_ids.shape[1], device).unsqueeze(0) + create_padding_mask(pl_ids, pl_tokenizer.token_to_id("[pad]"))
            outputs = model(en_ids, pl_ids, src_mask, tgt_mask)
            loss = loss_fn(outputs[:, :-1, :].reshape(-1, outputs.shape[-1]), pl_ids[:, 1:].reshape(-1))
            epoch_loss += loss.item()
    print(f"Eval loss: {epoch_loss/len(dataloader)}")
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), f"transformer-epoch-{epoch+1}.pth")

/home/machineblue/repositories/translator/.venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 1/60; Avg loss 4.096698644128886; Latest loss 3.0250887870788574


Evaluating: 100%|██████████| 1622/1622 [00:13<00:00, 117.79it/s]


Eval loss: 2.8337806796175045
Epoch 2/60; Avg loss 2.788626352309005; Latest loss 2.6558470726013184


Evaluating: 100%|██████████| 1622/1622 [00:13<00:00, 117.27it/s]


...

Eval loss: 0.03411191303263454
Epoch 59/60; Avg loss 0.034735216888623216; Latest loss 0.006583702750504017


Evaluating: 100%|██████████| 1622/1622 [00:13<00:00, 117.30it/s]


Eval loss: 0.033753974537184844
Epoch 60/60; Avg loss 0.033859962240922574; Latest loss 0.03819034621119499


Evaluating: 100%|██████████| 1622/1622 [00:13<00:00, 118.12it/s]

Eval loss: 0.03395433518871936


In [20]:
# Test for a few samples
model.eval()
N_SAMPLES = 5
MAX_LEN = 60
with torch.no_grad():
    start_token = torch.tensor([pl_tokenizer.token_to_id("[start]")]).to(device)
    for en, true_fr in random.sample(dataset.text_pairs, N_SAMPLES):
        en_ids = torch.tensor(en_tokenizer.encode(en).ids).unsqueeze(0).to(device)

        # get context from encoder
        src_mask = create_padding_mask(en_ids, en_tokenizer.token_to_id("[pad]"))
        x = model.src_embedding(en_ids)
        for encoder in model.encoders:
            x = encoder(x, src_mask, model.rope)
        enc_out = x

        # generate output from decoder
        pl_ids = start_token.unsqueeze(0)
        for _ in range(MAX_LEN):
            tgt_mask = create_causal_mask(pl_ids.shape[1], device).unsqueeze(0)
            tgt_mask = tgt_mask + create_padding_mask(pl_ids, pl_tokenizer.token_to_id("[pad]"))
            x = model.tgt_embedding(pl_ids)
            for decoder in model.decoders:
                x = decoder(x, enc_out, tgt_mask, model.rope)
            outputs = model.out(x)

            outputs = outputs.argmax(dim=-1)
            pl_ids = torch.cat([pl_ids, outputs[:, -1:]], axis=-1)
            if pl_ids[0, -1] == pl_tokenizer.token_to_id("[end]"):
                break

        # Decode the predicted IDs
        pred_fr = pl_tokenizer.decode(pl_ids[0].tolist())
        print(f"English: {en}")
        print(f"Polish: {true_fr}")
        print(f"Predicted: {pred_fr}")
        print()

English: i'm saving money in order to study abroad.
Polish: oszczędzam pieniądze na studia za granicą.
Predicted:  oszczędzam pieniądze na studia za granicą. 

English: you're too young to get married, aren't you?
Polish: jesteś za młoda żeby wyjść za mąż, nieprawdaż?
Predicted:  jesteś za młoda żeby wyjść za mąż, nieprawdaż? 

English: what's your favorite swear word?
Polish: jakie jest twoje ulubione przekleństwo?
Predicted:  co jest ukratne coś do pożyte? 

English: we've got so much to talk about.
Polish: mamy sporo do pogadania.
Predicted:  mamy sporo do pogadania. 

English: i just don't want to have people thinking i'm weak.
Polish: po prostu nie chcę, aby ludzie myśleli, że jestem słaby.
Predicted:  po prostu nie chcę, aby ludzie myśleli, że jestem słaby. 

